In [ ]:
# | eval: false
import warnings

import datasets
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import polars as pl
import seaborn as sns
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer

warnings.filterwarnings("ignore")
sns.set_theme("notebook")
transformers.logging.set_verbosity_error()
datasets.logging.set_verbosity_error()
datasets.utils.disable_progress_bars()

In [ ]:
# | eval: false
raw_ds = pl.scan_parquet(source="../../data/datasets/pl/raw/*")

# Dataset Card for [JuDDGES/pl-court-raw](https://huggingface.co/datasets/JuDDGES/pl-court-raw)

## Table of Contents
- [Table of Contents](#table-of-contents)
- [Dataset Description](#dataset-description)
  - [Dataset Summary](#dataset-summary)
  - [Supported Tasks and Leaderboards](#supported-tasks-and-leaderboards)
  - [Languages](#languages)
- [Dataset Structure](#dataset-structure)
  - [Data Instances](#data-instances)
  - [Data Fields](#data-fields)
  - [Data Splits](#data-splits)
- [Dataset Creation](#dataset-creation)
  - [Curation Rationale](#curation-rationale)
  - [Source Data](#source-data)
  - [Annotations](#annotations)
  - [Personal and Sensitive Information](#personal-and-sensitive-information)
- [Considerations for Using the Data](#considerations-for-using-the-data)
  - [Social Impact of Dataset](#social-impact-of-dataset)
  - [Discussion of Biases](#discussion-of-biases)
  - [Other Known Limitations](#other-known-limitations)
- [Additional Information](#additional-information)
  - [Dataset Curators](#dataset-curators)
  - [Licensing Information](#licensing-information)
  - [Citation Information](#citation-information)
  - [Contributions](#contributions)
- [Statistics](#statistics)

## Dataset Description

* **Homepage: TBA**
* **Repository: https://github.com/pwr-ai/JuDDGES**
* **Paper:  TBA**
* **Point of Contact: lukasz.augustyniak@pwr.edu.pl; jakub.binkowski@pwr.edu.pl; albert.sawczyn@pwr.edu.pl**

### Dataset Summary

The dataset consists of Polish Court judgements available at https://orzeczenia.ms.gov.pl/, containing full content of the judgements along with metadata sourced from official API and extracted from the judgement contents. This dataset contains raw data. For instruction dataset see [`JuDDGES/pl-court-instruct`](https://huggingface.co/datasets/JuDDGES/pl-court-instruct). For graph dataset see [`JuDDGES/pl-court-graph`](https://huggingface.co/datasets/JuDDGES/pl-court-graph).

### Supported Tasks and Leaderboards

The dataset can be used for various tasks. However, it contains raw data acquired from official API, and we rather recommend using instruction dataset [`JuDDGES/pl-court-instruct`](https://huggingface.co/datasets/JuDDGES/pl-court-instruct) for straightforward usage. 

### Languages

pl-PL Polish 

## Dataset Structure

### Data Instances

<details>
<summary> Click to expand </summary>

```
{'_id': '154505000005127_XVII_AmA_000014_1994_Uz_1994-07-20_001',
 'signature': 'XVII AmA 14/94',
 'date': '1994-07-20 02:00:00.0 CEST',
 'publicationDate': '2018-06-13 22:10:05.0 CEST',
 'lastUpdate': '2018-06-13 16:22:22.0 CEST',
 'courtId': '15450500',
 'departmentId': '5127',
 'type': 'SENTENCE, REASON',
 'excerpt': 'Sygn. akt. XVII Amr 14/94 WYROK W IMIENIU RZECZYPOSPOLITEJ POLSKIEJ Dnia 20 lipca 1994 r. Sąd Wojewódzki w Warszawie - Sąd Antymonopolowy w składzie następującym: Przewodniczący SSW Stanisław Gronowski (spr.) Sędziowie: SW Jacek Sikorski i SR Bogdan Gierzyński Protokólant apl. rade. A. Grześkiewicz po rozpoznaniu w dniu 6 lipca 1994 r. w Warszawie na rozpra\xadwie spraw z odwołań: A. M. , B. U. Sklep (...) we W. , Przedsiębiorstwa Handlowego (...) s.c. we W. , W. K. i A. B. (...) we W. od decyzji U',
 'content': '<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<xPart xPublisherFullName="Beata Gonera" xPublisher="bgonera" xVersion="1.0" xYear="1994" xLang="PL" xToPage="7" xEditor="bgonera" xEditorFullName="Beata Gonera" xFlag="published" xDocType="Uz" xml:space="preserve" xFromPg="1" xVolType="15/450500/0005127/AmA" xVolNmbr="000014">\n  <xName>Wyrok+Uzasadnienie</xName>\n  <xBlock>\n    <xText>Sygn. akt. XVII Amr 14/94</xText>\n    <xUnit xIsTitle="true" xBold="true" xType="part">\n      <xName>WYROK</xName>\n      <xTitle>W IMIENIU RZECZYPOSPOLITEJ POLSKIEJ</xTitle>\n      <xText xALIGNx="right">Dnia 20 lipca 1994 r.</xText>\n      <xText>Sąd Wojewódzki w Warszawie - Sąd Antymonopolowy w składzie następującym:</xText>\n      <xText>Przewodniczący SSW Stanisław Gronowski (spr.)</xText>\n      <xText>Sędziowie: SW Jacek Sikorski i SR Bogdan Gierzyński</xText>\n      <xText>Protokólant apl. rade. A. Grześkiewicz</xText>\n      <xText>po rozpoznaniu w dniu 6 lipca 1994 r. w Warszawie na rozpra\xadwie</xText>\n      <xText>spraw z odwołań: <xAnon>A. M.</xAnon>, <xAnon>B. U.</xAnon></xText>\n      <xText><xAnon> Sklep (...)</xAnon> we <xAnon>W.</xAnon>, <xAnon> Przedsiębiorstwa Handlowego (...) s.c.</xAnon> we <xAnon>W.</xAnon>, <xAnon>W. K.</xAnon> i <xAnon>A. B.</xAnon> <xAnon> (...)</xAnon> we <xAnon>W.</xAnon></xText>\n      <xText>od decyzji Urzędu Antymonopo1owego Delegatury we <xAnon>W.</xAnon> z dnia 29 października 1993 r., sygn. akt <xAnon>(...)</xAnon> przeciwko Urzędowi Antymonopolowemu zainteresowany: Gmina <xAnon>W.</xAnon></xText>\n      <xText>o przeciwdziałanie praktykom monopolistycznym</xText>\n      <xUnit xIsTitle="false" xType="none">\n        <xName xSffx=".">1</xName>\n        <xText>odwołania oddala</xText>\n      </xUnit>\n      <xUnit xIsTitle="false" xType="none">\n        <xName xSffx=".">2</xName>\n        <xText>zasądza na rzecz Urzędu Antymonopolowego od:</xText>\n        <xUnit xIsTitle="false" xType="none">\n          <xName xSffx=")">a</xName>\n          <xText><xAnon>A. M.</xAnon> kwotę 5.000.000 (pięć milionów) złotych</xText>\n        </xUnit>\n        <xUnit xIsTitle="false" xType="none">\n          <xName xSffx=")">b</xName>\n          <xText><xAnon>B. U.</xAnon> <xAnon> Sklep (...)</xAnon> we <xAnon>W.</xAnon> kwotę 5.000.000 (pięć milionów) złotych</xText>\n        </xUnit>\n        <xUnit xIsTitle="false" xType="none">\n          <xName xSffx=")">c</xName>\n          <xText><xAnon> Przedsiębiorstwa Handlowego (...) s.c.</xAnon> we <xAnon>W.</xAnon> kwotę 5.000.000 (pięć milionów) złotych</xText>\n        </xUnit>\n      </xUnit>\n      <xText><xAnon>(...)</xAnon>-</xText>\n      <xUnit xIsTitle="false" xType="none">\n        <xName xSffx=")">d</xName>\n        <xText>\n  <xAnon>W. K.</xAnon>i <xAnon>A. B.</xAnon> <xAnon>(...)</xAnon> s.c. we <xAnon>W.</xAnon>kwotę 5.000.000 (pięć milionów) złotych tytułem kosztów postępowania.</xText>\n      </xUnit>\n    </xUnit>\n    <xUnit xIsTitle="true" xBold="true" xType="part">\n      <xName>UZASADNIENIE</xName>\n      <xText>Gmina <xAnon>W.</xAnon> (zainteresowana w postępowaniu przed Sądem Antymonopolowym w rozumieniu <xLexLink xArt="art. 479(31);art. 479(31) § 2" xIsapId="WDU19640430296" xTitle="Ustawa z dnia 17 listopada 1964 r. - Kodeks postępowania cywilnego" xAddress="Dz. U. z 1964 r. Nr 43, poz. 296">art. 479<xSUPx>(\n    \n    \n    31)</xSUPx>§ 2 k.p.c.</xLexLink>) jest wła\xadścicielem lokali użytkowych zlokalizowanych na terenie miasta <xAnon>W.</xAnon>, które wynajmuje podmiotom gospodarczym. Zadecydo\xadwała, aby najem wspomnianych lokali był poprzedzony procedurą przetargową. W tym przedmiocie podjęta została uchwała Nr <xAnon>(...)</xAnon> Rady Miejskiej we <xAnon>W.</xAnon> z dnia 20 lutego 1993 r. w sprawie zasad wynajmowania lokali użytkowych stanowią\xadcych własność Gminy <xAnon>W.</xAnon>. Przetargami, w świetle §3 uchwały, objęto w szczególności lokale użytkowe przeznaczone na działalność handlową, produkcyjną, usługową i gastronomi\xadczną położone w centrum handlowym<xAnon>W.</xAnon>. Wspomniana pro\xadcedura obejmowała także lokale oddane pomiotom gospodarczym w następstwie działań prywatyzujących handel, usługi i gastro\xadnomię, jakie miały miejsce w 1990 r.</xText>\n      <xText>Z najemcami lokali przewidzianych do przetargu, do czasu ich przeprowadzenia, zawierano w miesiącach 1istopad-grudzień 1990 r. umowy najmu według ramowego wzoru. Umowa była zawie\xadrana na okres 1,5 roku, a więc na czas ściśle określony (§ 12). Po upływie tego terminu dotychczasowy najemca nadal uży\xadtkował lokal, jednakże musiał się liczyć z koniecznością jego natychmiastowego opróżnienia, jeżeli nie wygrał przetargu. Umowa zawierała zapisy dotyczące możliwości jej zmiany. W świetle § 17 ust. 1 wszelkie zmiany warunków umowy wymagają</xText>\n      <xText xALIGNx="center">- 3</xText>\n      <xText>formy pisemnej w postaci aneksu. Wyjątek stanowią zmiany w wysokości czynszu i innych opłat, o których najemca będzie powiadomiony odrębnym pismem (§ 17 ust. 2 umowy). Ponadto, jeżeli w czasie trwania stosunku najmu nastąpi zmiana okoli\xadczności mających wpływ na wysokość czynszu najmu, wynajmujący określa wysokość czynszu stosownie do zmienionych okolicznoś\xadci (§ 17 ust. 3 umowy). Dotychczasowi najemcy stawali do przetargu i w około 65 % przetarg kończył się dla nich pomyś\xadlnie. Z najemcami tymi były zawierane umowy na okres 5 lat.</xText>\n      <xText>Najemcy, w tym m. in. <xAnon>B. U.</xAnon>, Przedsiębiors\xadtwo Handlowe <xAnon>(...)</xAnon> s.c. we <xAnon>W.</xAnon>, <xAnon>W. K.</xAnon> i <xAnon>A. B.</xAnon> "<xAnon>(...)</xAnon>" s.c. we \'<xAnon>W.</xAnon> i <xAnon>A. M.</xAnon> (powo\xaddowie w postępowaniu przed Sądem Antymonopolowym) upatrywali w postanowieniach §§ 12 i 17 umowy nadużywanie pozycji domi\xadnującej na rynku lokali użytkowych przez Gminę <xAnon>W.</xAnon>. Za\xadrzucali również kilkakrotne podwyższanie czynszu i ustalenie go na zbyt wysokim poziomie. Wskazywali także, iż po upływie umownego okresu, na jaki zawarto umowę najmu, gmina ustala wysokie opłaty za bezumowne korzystanie z lokalu. Domagali się od Urzędu Antymonopolowego (pozwany w postępowaniu przed Sądem Antymonopolowym) wszczęcia przeciwko Gminie <xAnon>W.</xAnon> po\xadstępowań i administracyjnego.</xText>\n      <xText>Najemcy wnosili o nakazanie gminie zaniechania stosowa\xadnia praktyk monopolistycznych polegających na nadużywaniu po\xadzycji dominującej na rynku przez:</xText>\n      <xText>- jednostronne ustalanie zawyżonych stawek czynszu najmu lokali użytkowych z pominięciem trybu renegocjowania wa\xadrunków umowy z dotychczasowymi użytkownikami,</xText>\n      <xEnum>\n        <xBullet>-</xBullet>\n        <xEnumElem>\n          <xText>zawieranie krótkotrwałych umów najmu oraz udostępnianie lo\xadkali po upływie okresu umownego dotychczasowym najemcom na warunkach bezumownego korzystania z tychże lokali,</xText>\n        </xEnumElem>\n      </xEnum>\n      <xText>- 4-</xText>\n      <xEnum>\n        <xBullet>-</xBullet>\n        <xEnumElem>\n          <xText>przekazywanie do wynajęcia lokali w trybie przetargowym bez uprzedniego porozumienia się z dotychczasowymi najemcami co do warunków kontynuowania stosunku najmu.</xText>\n        </xEnumElem>\n      </xEnum>\n      <xText>Najemcy domagali się także, aby Urząd Antymonopolowy na\xadkazał gminie renegocjację warunków umowy najmu w części doty\xadczącej terminu obowiązywania umowy oraz zasad określenia sta\xadwek czynszu.</xText>\n      <xText>Urząd Antymonopolowy wszczął postępowanie administracyj\xadne w sprawie naruszenia przez gminę <xLexLink xArt="art. 4;art. 4 ust. 1;art. 4 ust. 1 pkt. 1;art. 5" xIsapId="WDU19900140088" xTitle="Ustawa z dnia 24 lutego 1990 r. o przeciwdziałaniu praktykom monopolistycznym" xAddress="Dz. U. z 1990 r. Nr 14, poz. 88">art. 4 ust. 1 pkt 1 i art. 5 ustawy z dnia 24 lutego 1990 r. o przeciwdziałaniu praktykom monopolistycznym</xLexLink> (Dz. U. z 1991 r. Nr 89, poz. 403), zwana dalej ustawą antymonopolową. W świetle art. 4 ust. 1 pkt 1 w/w ustawy praktyką monopolistyczną jest narzu\xadcanie uciążliwych warunków umów, przynoszących podmiotowi go\xadspodarczemu narzucającemu te warunki nieuzasadnione korzyści. Natomiast stosownie do art. 5 ustawy antymonopolowej praktyką monopolistyczną jest nadużywanie pozycji dominującej na ryn\xadku. Ponadto na okoliczność zawyżania czynszu Urząd Antymono\xadpolowy prowadził postępowanie z punktu widzenia naruszenia przez Gminę <xAnon>W.</xAnon> art. 7 ust. 1 pkt 3 w związku z art. 7 ust. 2 ustawy antymonopolowej. W świetle tych przepisów prak\xadtyką monopolistyczną jest pobieranie nadmiernie wygórowanych cen przez podmiot gospodarczy o utrwalonej pozycji dominują\xadcej na rynku.</xText>\n      <xText>W związku z zaskarżeniem przez najemców uchwały Rady Miejskiej <xAnon>W.</xAnon> z dnia 20 lutego 1993 r. Nr <xAnon>(...)</xAnon> do Naczelnego Sądu Administracyjnego sąd ten ustalił, że gmina zmierza drogą przetargu podwyższać czynsz najmu, w tym także w odniesieniu do osób z którymi zawarto umowy na czas okreś\xadlony, co stanowi działalność niezgodną z prawem. Tryb przeta\xadrgu nie może być wykorzystywany do zmiany wiążących umów naj-</xText>\n      <xText>- 5-</xText>\n      <xText>mu bądź ich rozwiązywania. W rezultacie wyrokiem z dnia 3 września 1993 r., sygn. akt SA/Wr/821-751-1236-1339/93 NSA</xText>\n      <xText>stwierdził nieważność § 3 wspomnianej uchwały.</xText>\n      <xText>W trakcie postępowania administracyjnego przed Urzędem Antymonopolowym gmina nie zaprzeczyła, udokumentowanym zresz\xadtą przez najemców faktom, stanowiącym podstawę żądania wszczęcia postępowania. Według gminy stosuje stawki opłat czynszowych ustalone uchwałą Zarządu Miasta <xAnon>W.</xAnon>. Stawki opłat czynszowych są zróżnicowane w zależności od strefy (po\xadłożenia lokalu) oraz branży od 150 tys./m<xSUPx>2</xSUPx> do 300 tys./m<xSUPx>2</xSUPx>. W następstwie przetargów gmina uzyskała czynsze od 300 tys./m<xSUPx>2</xSUPx> do 700 tys./m<xSUPx>2</xSUPx>. Deklarowała, iż najemcom którzy przegrali przetargi przygotuje lokale zamienne dla kontynuowania dzia\xadłalności gospodarczej.</xText>\n      <xText>Według ustaleń Urzędu Antymonopolowego na lokalnym rynku lokali użytkowych gmina posiada udział wynoszący około 80%. Posiada zatem pozycję dominującą w rozumieniu art. 2 pkt 7 ustawy antymonpolowej. Nadużyciem pozycji dominującej na ryn\xadku, godzącym w zasadę pewności obrotu, było bezumowne udostę\xadpnianie przez gminę lokali użytkowych podmiotom gospodarczym, zamiast zawieranie z nimi umów najmu. Urząd Antymonopolowy nakazał zatem gminie zaniechanie stosowania tej praktyki mo\xadnopolistycznej (pkt I decyzji z dnia 29 października 1993 r. nr <xAnon>(...)</xAnon>).</xText>\n      <xText>Ustosunkowując się do zarzutów najemców w przedmiocie zawierania z nimi przez gminę umów na okres 1,5 roku Urząd Antymonopolowy wyraził pogląd, iż jest to element, nie kwes\xadtionowanej przez niego, polityki gminy w zakresie oddawania lokali po uprzednim przeprowadzeniu przetargu. Do czasu okre\xadślenia zasad przeznaczenia lokali do przetargu gmina musiała zawierać z najemcami takie umowy. Stąd też Urząd Antymonopo-</xText>\n      <xText xALIGNx="center">- 6 -</xText>\n      <xText>Iowy nie dopatrzył się nadużycia pozycji dominującej ze stro\xadny gminy w zawieraniu umów na czas oznaczony, tym bardziej że <xLexLink xArt="" xIsapId="WDU19640160093" xTitle="Ustawa z dnia 23 kwietnia 1964 r. - Kodeks cywilny" xAddress="Dz. U. z 1964 r. Nr 16, poz. 93">Kodeks cywilny</xLexLink> nie zawiera ograniczeń co do długości trwania najmu. Urząd Antymonopolowy nie podzielił także stanowiska najemców zarzucających gminie nadużywanie pozycji dominującej na rynku przez jednostronne ustalanie zawyżonych stawek czyn\xadszu najmu. Porównując stawki czynszowe stosowane przez gminę ze stawkami innych podmiotów gospodarczych wynajmujących lo\xadkale użytkowe Urząd Antymonopolowy ustalił, iż gmina stosuje w tym względzie stawki na średnim poziomie. Uznał za dopusz\xadczalne w świetle przepisów <xLexLink xArt="" xIsapId="WDU19640160093" xTitle="Ustawa z dnia 23 kwietnia 1964 r. - Kodeks cywilny" xAddress="Dz. U. z 1964 r. Nr 16, poz. 93">Kodeksu cywilnego</xLexLink> jednostronne określanie przez wynajmującego wysokości czynszu bez zmiany samej umowy. W rezultacie więc kwestionowany przez najemców § 17 umowy uznał za nie naruszający prawa. Reasumując w pkt II decyzji Urząd Antymonopolowy nie stwierdził stosowania przez gminę praktyk monopolistycznych w zakresie nadużywania pozy\xadcji dominującej na rynku poprzez zawyżanie stawek czynszowych oraz zawieranie krótkoterminowych umów najmu. Jak to ustalił przy tym Urząd Antymonopolowy przeszkodą w ewentualnym kwes\xadtionowaniu tej części umowy, z punktu widzenia przepisów ustawy antymonopolowej, jest niestosowanie przez gminę wspom\xadnianych praktyk już od blisko 3 lat. W świetle art. S ust. 1 ustawy antymonopolowej nie wydaje się decyzji stwierdzającej stosowanie praktyk monopolistycznych i nakazującej ich zanie\xadchanie, jeżeli od końca roku, w którym zaprzestano stosowania praktyk monopolistycznych, upłynął rok.</xText>\n      <xText>Według Urzędu Antymonopolowego uchwała Rady Gminy <xAnon>W.</xAnon>, ustalająca wymóg przetargów}\' w odniesieniu do lokali użytkowych oddawanych w najem, nie podlega ocenie przepisów ustawy antymonopolowej. Wspomniana uchwała może być obalona jedynie w trybie przepisów ustawy o samorządzie terytorial-</xText>\n      <xText>nym. Stąd też Urząd Antymonopolowy na podstawie <xLexLink xArt="art. 10/5;art. 10/5 § 1" xIsapId="WDU19600300168" xTitle="Ustawa z dnia 14 czerwca 1960 r. - Kodeks postępowania administracyjnego" xAddress="Dz. U. z 1960 r. Nr 30, poz. 168">art. 10.5 §1 k.p.a.</xLexLink> umorzył postępowanie w sprawie przeciwko gminie w za\xadkresie przeznaczenia lokali użytkowych do przetargu w centrum handlowym <xAnon>W.</xAnon> (pkt III decyzji).</xText>\n      <xText>Od decyzji Urzędu Antymonopolowego najemcy odwołali się do Sądu Antymonopolowego.</xText>\n      <xText>Pełnomocnik procesowy <xAnon>A. M.</xAnon> zaskarżył pkt II i III decyzji. Wystąpił w swym odwołaniu także poza przedmiot prowadzonego postępowania administracyjnego. Odwołanie upa\xadtruje bowiem w odmowie gminy sprzedaży lokali użytkowych ograniczenia wolnej konkurencji na rynku, co uzasadnia zarzut naruszenia art. 5 pkt 1 ustawy antymonopolowej. Według odwo\xadłania niezakwestionowane przez Urząd Antymonopolowy różnico\xadwanie przez gminę warunków umów lokali użytkowych w centrum handlowym <xAnon>W.</xAnon>i poza nim uzasadnia postawienie zaskar\xadżonej decyzji zarzutu naruszenia art. 5 pkt 2 ustawy antymo\xadnopolowej. W świetle tego przepisu praktyką monopolistyczną jest nadużycie pozycji dominującej w następstwie podziału ry\xadnku według kryteriów terytorialnych. Podtrzymuje też dotych\xadczasowe stanowisko prezentowane w postępowaniu administracyj\xadnym, iż praktyką monopolistyczną jest jednostronne dyktowanie przez gminę zawyżonych opłat czynszowych, jak również zawie\xadranie krótkoterminowych umów najmu. Polemizuje ze stanowis\xadkiem zajętym w pkt III zaskarżonej decyzji, iż Urząd Antymo\xadnopolowy nie może ingerować w sposób wykonywania przez gminę przysługujących jej praw własności do sporem objętych lokali użytkowych.</xText>\n      <xText>Odwołanie wniesione przez pozostałych najemców, aczkol\xadwiek formalnie ograniczone do pktu III decyzji, zarzuca za\xadskarżonej decyzji niedostrzeganie praktyk monopolistycznych w działalności gminy w zakresie prowadzenia przetargów lokali w</xText>\n      <xText xALIGNx="center">- 8</xText>\n      <xText>stosunku do których przysługują podmiotom gospodarczym prawa wynikające z umowy najmu. Odwołanie wywodzi, iż po upływie okresu najmu przyzwolenie gminy na dalsze używanie lokali re\xadaktywuje urnowe najmu. Gmina powinna zatem najpierw uregulować stosunki łączące je z dotychczasowymi najemcami, a dopiero następnie skierować wolne lokale do przetargu. W tej sytuacji umorzenie w zaskarżonej decyzji postępowania w zakresie prze\xadznaczenia lokali do przetargu narusza <xLexLink xArt="art. 105;art. 105 § 1" xIsapId="WDU19600300168" xTitle="Ustawa z dnia 14 czerwca 1960 r. - Kodeks postępowania administracyjnego" xAddress="Dz. U. z 1960 r. Nr 30, poz. 168">art. 105 § 1 k.p.a.</xLexLink></xText>\n      <xText>Sąd Antymonopolowy zważył, co następuje:</xText>\n      <xText>Umowy są podstawowym instrumentem prawnym organizującym działalność pomiędzy podmiotami gospodarczymi w warunkach go\xadspodarki rynkowej. Zgodnie z założeniami takiej gospodarki strony mają swobodę wyboru kontrahenta, zaś istotne elementy treści umowy, w szczególności odnoszące się do ekwiwalentnoś\xadci świadczeń, kształtują wolne od zewnętrznych wpływów mecha\xadnizmy rynkowe. Pozwala to w sposób optymalny kształtować pra\xadwa i obowiązki stron i przeciwdziała zjawisku dyktowania tre\xadści umów przez podmiot silniejszy ekonomicznie. Wspomniane założenia mogą być jednakże realizowane jedynie w sytuacji rozbudowanego rynku, a w szczególności funkcjonowania po stronie podaży znacznej liczby podmiotów gospodarczych, kon\xadkurujących ze sobą. W okolicznościach faktycznych niniejszej sprawy taka sytuacja nie zachodzi. Gmina na rynku lokali uży\xadtkowych posiada bowiem silnie dominującą pozycję, zbliżoną do monopolistycznej. Pozwala to dyktować jej treść umów działa\xadjącym w rozproszeniu najemcom.</xText>\n      <xText>W celu przeciwdziałania nadużywaniu władzy rynkowej, a w szczególności tam, gdzie nie mogą funkcjonować mechanizmy ekonomiczne (z uwagi na szczupły rynek), umowy organizujące działalność gospodarczą powinny podlegać reglamentacji z <xLexLink xArt="" xIsapId="WDU20030600535" xTitle="Ustawa z dnia 28 lutego 2003 r. - Prawo upadłościowe i naprawcze" xAddress="Dz. U. z 2003 r. Nr 60, poz. 535">pun</xLexLink>\xadktu widzenia przestrzegania przepisów ustawy antymonopolowej.</xText>\n      <xText xALIGNx="center">- 9</xText>\n      <xText>Należy podkreślić, że zasada swobody umów wyrażona w <xLexLink xArt="art. 353(1)" xIsapId="WDU19640160093" xTitle="Ustawa z dnia 23 kwietnia 1964 r. - Kodeks cywilny" xAddress="Dz. U. z 1964 r. Nr 16, poz. 93">art. 353<xSUPx>1</xSUPx> k.c.</xLexLink> nie ma bezwzględnego charakteru. Treść umów nie mo\xadże bowiem sprzeciwiać się ustawom, w tym także przepisom ustawy antymonopolowej. Zgodnie z art. 8 ust. 2 w/w ustawy umowy zawarte z naruszeniem art. 4, art. 5 oraz art. 7 są w całości lub w odpowiedniej części nieważne. Wynika stąd, że ustawa antymonopolowa jest jedną z tych ustaw, do których od\xadsyła <xLexLink xArt="art. 353(1)" xIsapId="WDU19640160093" xTitle="Ustawa z dnia 23 kwietnia 1964 r. - Kodeks cywilny" xAddress="Dz. U. z 1964 r. Nr 16, poz. 93">art. 353<xSUPx>1</xSUPx> k.c.</xLexLink> dla określenia granic swobody umów. W świetle powyższego stanowiska Sąd Antymonopolowy nie podziela argumentacji zaskarżonej decyzji, nie dopatrującej się naru\xadszenia ustawy antymonopolowej w sytuacji gdy treść umów najmu mieści <xSUPx>;</xSUPx>się w granicach przepisów <xLexLink xArt="" xIsapId="WDU19640160093" xTitle="Ustawa z dnia 23 kwietnia 1964 r. - Kodeks cywilny" xAddress="Dz. U. z 1964 r. Nr 16, poz. 93">Kodeksu cywilnego</xLexLink>, normują\xadcych umowę najmu. Wręcz przeciwnie. Ustawa antymonopolowa po\xadwinna zapobiegać nadużywaniu kodeksowej zasady swobody umów przez podmiot posiadający władzę na rynku, tym bardziej zaś gdy narzuca on swym kontrahentom umowy naruszające przepisy wspomnianego kodeksu.</xText>\n      <xText>Według zaskarżonej decyzji nie naruszało ustawy antymo\xadnopolowej zawarcie przez gminę umów najmu na czas oznaczony wynoszący 1,5 roku, skoro <xLexLink xArt="" xIsapId="WDU19640160093" xTitle="Ustawa z dnia 23 kwietnia 1964 r. - Kodeks cywilny" xAddress="Dz. U. z 1964 r. Nr 16, poz. 93">Kodeks cywilny</xLexLink> nie zawiera ograni\xadczeń co do minimalnego okresu najmu. Z tym stanowiskiem trud\xadno się zgodzić. Lokale użytkowe, stanowiące przedmiot postę\xadpowania, służą najemcom do prowadzenia działalności gospodar\xadczej, przede wszystkim handlowej. Umowa najmu lokalu użytko\xadwego zawarta na okres 1,5 roku podważa sens działalności gos\xadpodarczej, gdyż czyni ją tymczasową. Najemca lokalu nie może planować swej działalności gospodarczej, a w szczególności czynić stosownych nakładów. Musi się realnie liczyć, iż po upływie tak zakreślonego terminu będzie zmuszony lokal opróż\xadnić. W ocenie Sądu Antymonopolowego wymuszenie na najemcach zawarcia umów najmu na okres 1,5 roku było ze strony gminy</xText>\n      <xText xALIGNx="center">- 10</xText>\n      <xText>nadużyciem pozycji dominującej na rynku w świetle art. 5 ustawy antymonopolowej. Tym sposobem gmina, nie licząc się z potrzebami najemców, którym narzucała niekorzystne warunki umów, eksploatowała silnie dominującą pozycję na rynku.</xText>\n      <xText>Także nadużyciem pozycji dominującej na rynku ze strony gminy, czego nie zauważono w zaskarżonej decyzji, a na co wskazywali najemcy, było wymuszenie w § 17 umowy najmu możli\xadwości jej wypowiedzenia, w szczególności zaś w zakresie wyso\xadkości czynszu. Tym sposobem, pomimo że umowa najmu zawarta była na czas oznaczony, gmina podwyższała czynsz najmu ponad kwotę ustaloną w umowie. Postanowienie § 17 umowy naruszało zresztą także i art. 68S <xLexLink xArt="" xIsapId="WDU19640160093" xTitle="Ustawa z dnia 23 kwietnia 1964 r. - Kodeks cywilny" xAddress="Dz. U. z 1964 r. Nr 16, poz. 93">k.c.</xLexLink> W świetle tego przepisu wypo\xadwiedzieć można jedynie umowę najmu zawartą na czas nie ozna\xadczony. Umowa zawarta na czas ściśle oznaczony nie podlega wypowiedzeniu.</xText>\n      <xText>Zasadniczo jedyną sankcją, która może być zastosowana w stosunku do podmiotu stosującego praktyki monopolistyczne, jest decyzja nakazująca ich zaniechanie (art. 8 ust. 1 usta\xadwy). Wydanie takiej decyzji, w razie jej nierespektowania, pozwala na uruchamianie kar pieniężnych (art. 15 ustawy). Za\xadstosowanie sankcji administracyjnej z art. 8 ust. 1 w/w ustawy jest jednakże ograniczone pod względem czasowym. Decyzja taka, na mocy wyraźnego zapisu ustawowego, nie może być wyda\xadna, jeżeli od końca roku, w którym zaprzestano stosowania praktyki monopolistycznej, upłynął rok. W odniesieniu do sto\xadsowanej przez gminę praktyki zawierania umów na okres 1,5 ro\xadku zaniechano tego z końcem 1990 r. Natomiast praktyka wypo\xadwiadania wspomnianych umów w części dotyczącej czynszu nie miała już miejsca począwszy od marca 1991 r. Z omawianych więc względów w dacie złożenia przez najemców w miesiącu kwietniu 1993 r. żądania wszczęcia postępowania administracy-\xad</xText>\n      <xText xALIGNx="center">- 11</xText>\n      <xText>jnego przeciwko gminie nie wchodziła już w grę możliwość wy\xaddania decyzji z art. 8 ust. 1 w/w ustawy w odniesieniu do wspomnianych praktyk monopolistycznych.</xText>\n      <xText>Rozpoznając w pozostałym zakresie odwołania wniesione przez najemców Sąd Antymonopolowy również nie znalazł podstaw do ich uwzględnienia. Nietrafne są w szczególności zarzuty odwołań, iż w zaskarżonej decyzji nie dostrzeżono praktyki monopolistycznej w prowadzeniu przez gminę przetargów w od\xadniesieniu do lokali, w stosunku do których najemcom przysłu\xadgują prawa wynikające z umowy najmu. Przeciwdziała tej prak\xadtyce pkt I zaskarżonej decyzji, gdzie nakazano gminie zanie\xadchanie odmowy zawierania umów najmu w formie pisemnej w od-r niesieniu do lokali udostępnianych najemcom na zasadzie bezu\xadmownego korzystania. Rozstrzygnięcie Urzędu Antymonopolowego zgodne jest tutaj z zasadą wyrażoną w <xLexLink xArt="art. 674" xIsapId="WDU19640160093" xTitle="Ustawa z dnia 23 kwietnia 1964 r. - Kodeks cywilny" xAddress="Dz. U. z 1964 r. Nr 16, poz. 93">art. 674 k.c.</xLexLink></xText>\n      <xText>Wprawdzie nie można podzielić poglądu zajętego w zaskar\xadżonej decyzji jakoby podejmowane przez gminę, w wykonaniu przepisów gminnych, działania zmierzające do kierowania loka\xadli użytkowych do przetargu nie podlegały ocenie ustawy anty\xadmonopolowej. Tym niemniej, zakładając uporządkowanie gospoda\xadrki lokalami użytkowymi w następstwie wykonania pktu I decy\xadzji, trudno dopatrzyć się w instytucji przetargu naruszenia przepisów ustawy antymonopolowej. Jeżeli do przetargu mogą na równych prawach przystąpić wszystkie zainteresowane podmioty gospodarcze, przetarg nie narusza celów ustawy antymonopolo\xadwej a w szczególności ochrony wolnej konkurencji.</xText>\n      <xText>Sąd Antymonopolowy nie podzielił stanowiska zajętego w odwołaniu <xAnon>A. M.</xAnon>, w świetle którego różnicowanie przez gminę warunków umów najmu lokali użytkowych zlokalizo\xadwanych w centrum handlowym miasta w porównaniu do pozostałych lokali, stanowi podział rynku według kryterium terytorialnego</xText>\n      <xText xALIGNx="center">\n  - 12\n</xText>\n      <xText>lub asortymentowego w rozumieniu art. 5 pkt 2 ustawy antymo\xadnopolowej. Rynek w ujęciu przedmiotowym odnosi się do wszyst\xadkich towarów jednego rodzaju, które poprzez szczególne właś\xadciwości, w szczególności zaś przeznaczenie, użytkowość i cenę odróżniają się od innych towarów w taki sposób, że nie ist\xadnieje możliwość dowolnej ich zamiany. Rynek pod względem przedmiotowym (asortymentowym) tworzą więc towary, które z punktu widzenia nabywcy charakteryzują się substytutywnością. Pomimo więc podobnego pod względem przedmiotowym asortymentu, lokali użytkowych zlokalizowanych w centrum handlowym miasta w porównaniu do innych lokali użytkowych w tym mieście nie można, z punktu widzenia ogółu nabywców (najemców), uznać za substytutywne. Możliwości prowadzenia działalności gospodar\xadczej w lokalach znajdujących się w centrum handlowym miasta sa zdecydowanie korzystniejsze. Istotne jest także i to, że w warunkach gospodarki rynkowej obiektywną prawidłowością są znacznie wyższe czynsze najmu lokali użytkowych zlokalizowa\xadnych w centrach handlowych miast w porównaniu do pozostałych lokali. Uzasadnia to traktowanie takich lokali użytkowych ja\xadko odrębnego rynku. Z tego też względu stosowanie przez gminę innej polityki najmu w odniesieniu do tej kategorii lokali nic może być uznane jako podział rynku, lecz jako działania na dwóch odrębnych rynkach.</xText>\n      <xText>Brak jest także podstaw do uwzględnienia zarzutu odwoła\xadnia upatrującego w odmowie sprzedaży lokali użytkowych ogra\xadniczanie przez gminę wolnej konkurencji na rynku w świetle art. 5 pkt 1 ustawy antymonopolowej, skoro nie stanowiło to przedmiotu rozstrzygnięcia zaskarżonej decyzji, a nawet przedmiotu żądania zgłoszonego we wniosku o wszczęcie postę\xadpowania. Formalnie rzecz biorąc nie ma przeszkód, aby wspom-\xad</xText>\n      <xText xALIGNx="center">\n  - 13\n</xText>\n      <xText>niany zarzut stanowił przedmiot odrębnego postępowania administracyjnego.</xText>\n      <xText>Nie znajdują także uzasadnienia, podnoszone w szczególności w postępowaniu administracyjnym, żądania skierowane pod adresem Urzędu Antymonopolowego, aby ten zobowiązał gminę do renegocjowania treści umów najmu z dotychczasowymi najemcami. W świetle art. 8 ust. 1 ustawy antymonopolowej takie oczeki\xadwania pod adresem organów działających w trybie wspomnianej ustawy, są za daleko idące. W postępowaniu administracyjnym toczącym się w trybie wspomnianej ustawy podmiotowi stosują\xadcemu praktyki monopolistyczne można jedynie nakazać ich za\xadniechanie. Nie można natomiast, w sposób pozytywny, nakazać podmiotowi spełnienie określonego zachowania mieszczącego się w granicach prawa. Reasumując zatem w trybie ustawy antymono\xadpolowej można wskazać podmiotowi gospodarczemu czego ma za\xadniechać. Natomiast nie wolno mu w prawnoprocesowych formach działania wskazać jak konkretnie ma się zachować w określonej sytuacji.</xText>\n      <xText>W tym stanie sprawy Sąd Antymonopolowy odwołania oddalił (<xLexLink xArt="art. 479" xIsapId="WDU19640430296" xTitle="Ustawa z dnia 17 listopada 1964 r. - Kodeks postępowania cywilnego" xAddress="Dz. U. z 1964 r. Nr 43, poz. 296">art. 479</xLexLink><xSUPx>3 1</xSUPx> <xLexLink xArt="§ 1" xIsapId="WDU19640430296" xTitle="Ustawa z dnia 17 listopada 1964 r. - Kodeks postępowania cywilnego" xAddress="Dz. U. z 1964 r. Nr 43, poz. 296">§ 1 k.p.c.</xLexLink>).</xText>\n      <xText>O kosztach postępowania orzeczono na zasadzie <xLexLink xArt="art. 98;art. 99" xIsapId="WDU19640430296" xTitle="Ustawa z dnia 17 listopada 1964 r. - Kodeks postępowania cywilnego" xAddress="Dz. U. z 1964 r. Nr 43, poz. 296">art. 98 i 99 k.p.c.</xLexLink></xText>\n    </xUnit>\n  </xBlock>\n</xPart>\n',
 'chairman': 'SSW Stanisław Gronowski',
 'decision': None,
 'judges': ['SW Jacek Sikorski i Bogdan Gierzyński',
  'SSW Stanisław Gronowski'],
 'legalBases': ['art. 6 ust. 1 pkt 1 ustawy ochronie konkurencji i konsumentów',
  'art. 106 ust. 1 pkt 1 ustawy ochronie konkurencji i konsumentów'],
 'publisher': 'Beata Gonera',
 'recorder': None,
 'references': ['Ustawa z dnia 17 listopada 1964 r. - Kodeks postępowania cywilnego (Dz. U. z 1964 r. Nr 43, poz. 296 - art. 479; art. 479(31); art. 479(31) § 2; art. 98; art. 99; § 1)',
  'Ustawa z dnia 14 czerwca 1960 r. - Kodeks postępowania administracyjnego (Dz. U. z 1960 r. Nr 30, poz. 168 - art. 10/5; art. 10/5 § 1; art. 105; art. 105 § 1)',
  'Ustawa z dnia 28 lutego 2003 r. - Prawo upadłościowe i naprawcze (Dz. U. z 2003 r. Nr 60, poz. 535 - )',
  'Ustawa z dnia 24 lutego 1990 r. o przeciwdziałaniu praktykom monopolistycznym (Dz. U. z 1990 r. Nr 14, poz. 88 - art. 4; art. 4 ust. 1; art. 4 ust. 1 pkt. 1; art. 5)',
  'Ustawa z dnia 23 kwietnia 1964 r. - Kodeks cywilny (Dz. U. z 1964 r. Nr 16, poz. 93 - art. 353(1); art. 674)'],
 'reviser': 'Beata Gonera',
 'themePhrases': ['Praktyki ograniczające konkurencję',
  'Kary pieniężne UOKIK'],
 'num_pages': 7.0,
 'text': 'Sygn. akt. XVII Amr 14/94\n\nWYROK\nW IMIENIU RZECZYPOSPOLITEJ POLSKIEJ\nDnia 20 lipca 1994 r.\nSąd Wojewódzki w Warszawie - Sąd Antymonopolowy w składzie następującym:\nPrzewodniczący SSW Stanisław Gronowski (spr.)\nSędziowie: SW Jacek Sikorski i SR Bogdan Gierzyński\nProtokólant apl. rade. A. Grześkiewicz\npo rozpoznaniu w dniu 6 lipca 1994 r. w Warszawie na rozpra\xadwie\nspraw z odwołań:A. M.,B. U.\nSklep (...)weW.,Przedsiębiorstwa Handlowego (...) s.c.weW.,W. K.iA. B.(...)weW.\nod decyzji Urzędu Antymonopo1owego Delegatury weW.z dnia 29 października 1993 r., sygn. akt(...)przeciwko Urzędowi Antymonopolowemu zainteresowany: GminaW.\no przeciwdziałanie praktykom monopolistycznym\n\n1\nodwołania oddala\n\n2\nzasądza na rzecz Urzędu Antymonopolowego od:\n\na\nA. M.kwotę 5.000.000 (pięć milionów) złotych\n\nb\nB. U.Sklep (...)weW.kwotę 5.000.000 (pięć milionów) złotych\n\nc\nPrzedsiębiorstwa Handlowego (...) s.c.weW.kwotę 5.000.000 (pięć milionów) złotych\n\n(...)-\n\nd\n\nW. K.iA. B.(...)s.c. weW.kwotę 5.000.000 (pięć milionów) złotych tytułem kosztów postępowania.\n\nUZASADNIENIE\nGminaW.(zainteresowana w postępowaniu przed Sądem Antymonopolowym w rozumieniuart. 479(\n\n    31)§ 2 k.p.c.) jest wła\xadścicielem lokali użytkowych zlokalizowanych na terenie miastaW., które wynajmuje podmiotom gospodarczym. Zadecydo\xadwała, aby najem wspomnianych lokali był poprzedzony procedurą przetargową. W tym przedmiocie podjęta została uchwała Nr(...)Rady Miejskiej weW.z dnia 20 lutego 1993 r. w sprawie zasad wynajmowania lokali użytkowych stanowią\xadcych własność GminyW.. Przetargami, w świetle §3 uchwały, objęto w szczególności lokale użytkowe przeznaczone na działalność handlową, produkcyjną, usługową i gastronomi\xadczną położone w centrum handlowymW.. Wspomniana pro\xadcedura obejmowała także lokale oddane pomiotom gospodarczym w następstwie działań prywatyzujących handel, usługi i gastro\xadnomię, jakie miały miejsce w 1990 r.\nZ najemcami lokali przewidzianych do przetargu, do czasu ich przeprowadzenia, zawierano w miesiącach 1istopad-grudzień 1990 r. umowy najmu według ramowego wzoru. Umowa była zawie\xadrana na okres 1,5 roku, a więc na czas ściśle określony (§ 12). Po upływie tego terminu dotychczasowy najemca nadal uży\xadtkował lokal, jednakże musiał się liczyć z koniecznością jego natychmiastowego opróżnienia, jeżeli nie wygrał przetargu. Umowa zawierała zapisy dotyczące możliwości jej zmiany. W świetle § 17 ust. 1 wszelkie zmiany warunków umowy wymagają\n- 3\nformy pisemnej w postaci aneksu. Wyjątek stanowią zmiany w wysokości czynszu i innych opłat, o których najemca będzie powiadomiony odrębnym pismem (§ 17 ust. 2 umowy). Ponadto, jeżeli w czasie trwania stosunku najmu nastąpi zmiana okoli\xadczności mających wpływ na wysokość czynszu najmu, wynajmujący określa wysokość czynszu stosownie do zmienionych okolicznoś\xadci (§ 17 ust. 3 umowy). Dotychczasowi najemcy stawali do przetargu i w około 65 % przetarg kończył się dla nich pomyś\xadlnie. Z najemcami tymi były zawierane umowy na okres 5 lat.\nNajemcy, w tym m. in.B. U., Przedsiębiors\xadtwo Handlowe(...)s.c. weW.,W. K.iA. B."(...)" s.c. we \'W.iA. M.(powo\xaddowie w postępowaniu przed Sądem Antymonopolowym) upatrywali w postanowieniach §§ 12 i 17 umowy nadużywanie pozycji domi\xadnującej na rynku lokali użytkowych przez GminęW.. Za\xadrzucali również kilkakrotne podwyższanie czynszu i ustalenie go na zbyt wysokim poziomie. Wskazywali także, iż po upływie umownego okresu, na jaki zawarto umowę najmu, gmina ustala wysokie opłaty za bezumowne korzystanie z lokalu. Domagali się od Urzędu Antymonopolowego (pozwany w postępowaniu przed Sądem Antymonopolowym) wszczęcia przeciwko GminieW.po\xadstępowań i administracyjnego.\nNajemcy wnosili o nakazanie gminie zaniechania stosowa\xadnia praktyk monopolistycznych polegających na nadużywaniu po\xadzycji dominującej na rynku przez:\n- jednostronne ustalanie zawyżonych stawek czynszu najmu lokali użytkowych z pominięciem trybu renegocjowania wa\xadrunków umowy z dotychczasowymi użytkownikami,\n\n-\n\nzawieranie krótkotrwałych umów najmu oraz udostępnianie lo\xadkali po upływie okresu umownego dotychczasowym najemcom na warunkach bezumownego korzystania z tychże lokali,\n\n- 4-\n\n-\n\nprzekazywanie do wynajęcia lokali w trybie przetargowym bez uprzedniego porozumienia się z dotychczasowymi najemcami co do warunków kontynuowania stosunku najmu.\n\nNajemcy domagali się także, aby Urząd Antymonopolowy na\xadkazał gminie renegocjację warunków umowy najmu w części doty\xadczącej terminu obowiązywania umowy oraz zasad określenia sta\xadwek czynszu.\nUrząd Antymonopolowy wszczął postępowanie administracyj\xadne w sprawie naruszenia przez gminęart. 4 ust. 1 pkt 1 i art. 5 ustawy z dnia 24 lutego 1990 r. o przeciwdziałaniu praktykom monopolistycznym(Dz. U. z 1991 r. Nr 89, poz. 403), zwana dalej ustawą antymonopolową. W świetle art. 4 ust. 1 pkt 1 w/w ustawy praktyką monopolistyczną jest narzu\xadcanie uciążliwych warunków umów, przynoszących podmiotowi go\xadspodarczemu narzucającemu te warunki nieuzasadnione korzyści. Natomiast stosownie do art. 5 ustawy antymonopolowej praktyką monopolistyczną jest nadużywanie pozycji dominującej na ryn\xadku. Ponadto na okoliczność zawyżania czynszu Urząd Antymono\xadpolowy prowadził postępowanie z punktu widzenia naruszenia przez GminęW.art. 7 ust. 1 pkt 3 w związku z art. 7 ust. 2 ustawy antymonopolowej. W świetle tych przepisów prak\xadtyką monopolistyczną jest pobieranie nadmiernie wygórowanych cen przez podmiot gospodarczy o utrwalonej pozycji dominują\xadcej na rynku.\nW związku z zaskarżeniem przez najemców uchwały Rady MiejskiejW.z dnia 20 lutego 1993 r. Nr(...)do Naczelnego Sądu Administracyjnego sąd ten ustalił, że gmina zmierza drogą przetargu podwyższać czynsz najmu, w tym także w odniesieniu do osób z którymi zawarto umowy na czas okreś\xadlony, co stanowi działalność niezgodną z prawem. Tryb przeta\xadrgu nie może być wykorzystywany do zmiany wiążących umów naj-\n- 5-\nmu bądź ich rozwiązywania. W rezultacie wyrokiem z dnia 3 września 1993 r., sygn. akt SA/Wr/821-751-1236-1339/93 NSA\nstwierdził nieważność § 3 wspomnianej uchwały.\nW trakcie postępowania administracyjnego przed Urzędem Antymonopolowym gmina nie zaprzeczyła, udokumentowanym zresz\xadtą przez najemców faktom, stanowiącym podstawę żądania wszczęcia postępowania. Według gminy stosuje stawki opłat czynszowych ustalone uchwałą Zarządu MiastaW.. Stawki opłat czynszowych są zróżnicowane w zależności od strefy (po\xadłożenia lokalu) oraz branży od 150 tys./m2do 300 tys./m2. W następstwie przetargów gmina uzyskała czynsze od 300 tys./m2do 700 tys./m2. Deklarowała, iż najemcom którzy przegrali przetargi przygotuje lokale zamienne dla kontynuowania dzia\xadłalności gospodarczej.\nWedług ustaleń Urzędu Antymonopolowego na lokalnym rynku lokali użytkowych gmina posiada udział wynoszący około 80%. Posiada zatem pozycję dominującą w rozumieniu art. 2 pkt 7 ustawy antymonpolowej. Nadużyciem pozycji dominującej na ryn\xadku, godzącym w zasadę pewności obrotu, było bezumowne udostę\xadpnianie przez gminę lokali użytkowych podmiotom gospodarczym, zamiast zawieranie z nimi umów najmu. Urząd Antymonopolowy nakazał zatem gminie zaniechanie stosowania tej praktyki mo\xadnopolistycznej (pkt I decyzji z dnia 29 października 1993 r. nr(...)).\nUstosunkowując się do zarzutów najemców w przedmiocie zawierania z nimi przez gminę umów na okres 1,5 roku Urząd Antymonopolowy wyraził pogląd, iż jest to element, nie kwes\xadtionowanej przez niego, polityki gminy w zakresie oddawania lokali po uprzednim przeprowadzeniu przetargu. Do czasu okre\xadślenia zasad przeznaczenia lokali do przetargu gmina musiała zawierać z najemcami takie umowy. Stąd też Urząd Antymonopo-\n- 6 -\nIowy nie dopatrzył się nadużycia pozycji dominującej ze stro\xadny gminy w zawieraniu umów na czas oznaczony, tym bardziej żeKodeks cywilnynie zawiera ograniczeń co do długości trwania najmu. Urząd Antymonopolowy nie podzielił także stanowiska najemców zarzucających gminie nadużywanie pozycji dominującej na rynku przez jednostronne ustalanie zawyżonych stawek czyn\xadszu najmu. Porównując stawki czynszowe stosowane przez gminę ze stawkami innych podmiotów gospodarczych wynajmujących lo\xadkale użytkowe Urząd Antymonopolowy ustalił, iż gmina stosuje w tym względzie stawki na średnim poziomie. Uznał za dopusz\xadczalne w świetle przepisówKodeksu cywilnegojednostronne określanie przez wynajmującego wysokości czynszu bez zmiany samej umowy. W rezultacie więc kwestionowany przez najemców § 17 umowy uznał za nie naruszający prawa. Reasumując w pkt II decyzji Urząd Antymonopolowy nie stwierdził stosowania przez gminę praktyk monopolistycznych w zakresie nadużywania pozy\xadcji dominującej na rynku poprzez zawyżanie stawek czynszowych oraz zawieranie krótkoterminowych umów najmu. Jak to ustalił przy tym Urząd Antymonopolowy przeszkodą w ewentualnym kwes\xadtionowaniu tej części umowy, z punktu widzenia przepisów ustawy antymonopolowej, jest niestosowanie przez gminę wspom\xadnianych praktyk już od blisko 3 lat. W świetle art. S ust. 1 ustawy antymonopolowej nie wydaje się decyzji stwierdzającej stosowanie praktyk monopolistycznych i nakazującej ich zanie\xadchanie, jeżeli od końca roku, w którym zaprzestano stosowania praktyk monopolistycznych, upłynął rok.\nWedług Urzędu Antymonopolowego uchwała Rady GminyW., ustalająca wymóg przetargów}\' w odniesieniu do lokali użytkowych oddawanych w najem, nie podlega ocenie przepisów ustawy antymonopolowej. Wspomniana uchwała może być obalona jedynie w trybie przepisów ustawy o samorządzie terytorial-\nnym. Stąd też Urząd Antymonopolowy na podstawieart. 10.5 §1 k.p.a.umorzył postępowanie w sprawie przeciwko gminie w za\xadkresie przeznaczenia lokali użytkowych do przetargu w centrum handlowymW.(pkt III decyzji).\nOd decyzji Urzędu Antymonopolowego najemcy odwołali się do Sądu Antymonopolowego.\nPełnomocnik procesowyA. M.zaskarżył pkt II i III decyzji. Wystąpił w swym odwołaniu także poza przedmiot prowadzonego postępowania administracyjnego. Odwołanie upa\xadtruje bowiem w odmowie gminy sprzedaży lokali użytkowych ograniczenia wolnej konkurencji na rynku, co uzasadnia zarzut naruszenia art. 5 pkt 1 ustawy antymonopolowej. Według odwo\xadłania niezakwestionowane przez Urząd Antymonopolowy różnico\xadwanie przez gminę warunków umów lokali użytkowych w centrum handlowymW.i poza nim uzasadnia postawienie zaskar\xadżonej decyzji zarzutu naruszenia art. 5 pkt 2 ustawy antymo\xadnopolowej. W świetle tego przepisu praktyką monopolistyczną jest nadużycie pozycji dominującej w następstwie podziału ry\xadnku według kryteriów terytorialnych. Podtrzymuje też dotych\xadczasowe stanowisko prezentowane w postępowaniu administracyj\xadnym, iż praktyką monopolistyczną jest jednostronne dyktowanie przez gminę zawyżonych opłat czynszowych, jak również zawie\xadranie krótkoterminowych umów najmu. Polemizuje ze stanowis\xadkiem zajętym w pkt III zaskarżonej decyzji, iż Urząd Antymo\xadnopolowy nie może ingerować w sposób wykonywania przez gminę przysługujących jej praw własności do sporem objętych lokali użytkowych.\nOdwołanie wniesione przez pozostałych najemców, aczkol\xadwiek formalnie ograniczone do pktu III decyzji, zarzuca za\xadskarżonej decyzji niedostrzeganie praktyk monopolistycznych w działalności gminy w zakresie prowadzenia przetargów lokali w\n- 8\nstosunku do których przysługują podmiotom gospodarczym prawa wynikające z umowy najmu. Odwołanie wywodzi, iż po upływie okresu najmu przyzwolenie gminy na dalsze używanie lokali re\xadaktywuje urnowe najmu. Gmina powinna zatem najpierw uregulować stosunki łączące je z dotychczasowymi najemcami, a dopiero następnie skierować wolne lokale do przetargu. W tej sytuacji umorzenie w zaskarżonej decyzji postępowania w zakresie prze\xadznaczenia lokali do przetargu naruszaart. 105 § 1 k.p.a.\nSąd Antymonopolowy zważył, co następuje:\nUmowy są podstawowym instrumentem prawnym organizującym działalność pomiędzy podmiotami gospodarczymi w warunkach go\xadspodarki rynkowej. Zgodnie z założeniami takiej gospodarki strony mają swobodę wyboru kontrahenta, zaś istotne elementy treści umowy, w szczególności odnoszące się do ekwiwalentnoś\xadci świadczeń, kształtują wolne od zewnętrznych wpływów mecha\xadnizmy rynkowe. Pozwala to w sposób optymalny kształtować pra\xadwa i obowiązki stron i przeciwdziała zjawisku dyktowania tre\xadści umów przez podmiot silniejszy ekonomicznie. Wspomniane założenia mogą być jednakże realizowane jedynie w sytuacji rozbudowanego rynku, a w szczególności funkcjonowania po stronie podaży znacznej liczby podmiotów gospodarczych, kon\xadkurujących ze sobą. W okolicznościach faktycznych niniejszej sprawy taka sytuacja nie zachodzi. Gmina na rynku lokali uży\xadtkowych posiada bowiem silnie dominującą pozycję, zbliżoną do monopolistycznej. Pozwala to dyktować jej treść umów działa\xadjącym w rozproszeniu najemcom.\nW celu przeciwdziałania nadużywaniu władzy rynkowej, a w szczególności tam, gdzie nie mogą funkcjonować mechanizmy ekonomiczne (z uwagi na szczupły rynek), umowy organizujące działalność gospodarczą powinny podlegać reglamentacji zpun\xadktu widzenia przestrzegania przepisów ustawy antymonopolowej.\n- 9\nNależy podkreślić, że zasada swobody umów wyrażona wart. 3531k.c.nie ma bezwzględnego charakteru. Treść umów nie mo\xadże bowiem sprzeciwiać się ustawom, w tym także przepisom ustawy antymonopolowej. Zgodnie z art. 8 ust. 2 w/w ustawy umowy zawarte z naruszeniem art. 4, art. 5 oraz art. 7 są w całości lub w odpowiedniej części nieważne. Wynika stąd, że ustawa antymonopolowa jest jedną z tych ustaw, do których od\xadsyłaart. 3531k.c.dla określenia granic swobody umów. W świetle powyższego stanowiska Sąd Antymonopolowy nie podziela argumentacji zaskarżonej decyzji, nie dopatrującej się naru\xadszenia ustawy antymonopolowej w sytuacji gdy treść umów najmu mieści;się w granicach przepisówKodeksu cywilnego, normują\xadcych umowę najmu. Wręcz przeciwnie. Ustawa antymonopolowa po\xadwinna zapobiegać nadużywaniu kodeksowej zasady swobody umów przez podmiot posiadający władzę na rynku, tym bardziej zaś gdy narzuca on swym kontrahentom umowy naruszające przepisy wspomnianego kodeksu.\nWedług zaskarżonej decyzji nie naruszało ustawy antymo\xadnopolowej zawarcie przez gminę umów najmu na czas oznaczony wynoszący 1,5 roku, skoroKodeks cywilnynie zawiera ograni\xadczeń co do minimalnego okresu najmu. Z tym stanowiskiem trud\xadno się zgodzić. Lokale użytkowe, stanowiące przedmiot postę\xadpowania, służą najemcom do prowadzenia działalności gospodar\xadczej, przede wszystkim handlowej. Umowa najmu lokalu użytko\xadwego zawarta na okres 1,5 roku podważa sens działalności gos\xadpodarczej, gdyż czyni ją tymczasową. Najemca lokalu nie może planować swej działalności gospodarczej, a w szczególności czynić stosownych nakładów. Musi się realnie liczyć, iż po upływie tak zakreślonego terminu będzie zmuszony lokal opróż\xadnić. W ocenie Sądu Antymonopolowego wymuszenie na najemcach zawarcia umów najmu na okres 1,5 roku było ze strony gminy\n- 10\nnadużyciem pozycji dominującej na rynku w świetle art. 5 ustawy antymonopolowej. Tym sposobem gmina, nie licząc się z potrzebami najemców, którym narzucała niekorzystne warunki umów, eksploatowała silnie dominującą pozycję na rynku.\nTakże nadużyciem pozycji dominującej na rynku ze strony gminy, czego nie zauważono w zaskarżonej decyzji, a na co wskazywali najemcy, było wymuszenie w § 17 umowy najmu możli\xadwości jej wypowiedzenia, w szczególności zaś w zakresie wyso\xadkości czynszu. Tym sposobem, pomimo że umowa najmu zawarta była na czas oznaczony, gmina podwyższała czynsz najmu ponad kwotę ustaloną w umowie. Postanowienie § 17 umowy naruszało zresztą także i art. 68Sk.c.W świetle tego przepisu wypo\xadwiedzieć można jedynie umowę najmu zawartą na czas nie ozna\xadczony. Umowa zawarta na czas ściśle oznaczony nie podlega wypowiedzeniu.\nZasadniczo jedyną sankcją, która może być zastosowana w stosunku do podmiotu stosującego praktyki monopolistyczne, jest decyzja nakazująca ich zaniechanie (art. 8 ust. 1 usta\xadwy). Wydanie takiej decyzji, w razie jej nierespektowania, pozwala na uruchamianie kar pieniężnych (art. 15 ustawy). Za\xadstosowanie sankcji administracyjnej z art. 8 ust. 1 w/w ustawy jest jednakże ograniczone pod względem czasowym. Decyzja taka, na mocy wyraźnego zapisu ustawowego, nie może być wyda\xadna, jeżeli od końca roku, w którym zaprzestano stosowania praktyki monopolistycznej, upłynął rok. W odniesieniu do sto\xadsowanej przez gminę praktyki zawierania umów na okres 1,5 ro\xadku zaniechano tego z końcem 1990 r. Natomiast praktyka wypo\xadwiadania wspomnianych umów w części dotyczącej czynszu nie miała już miejsca począwszy od marca 1991 r. Z omawianych więc względów w dacie złożenia przez najemców w miesiącu kwietniu 1993 r. żądania wszczęcia postępowania administracy-\xad\n- 11\njnego przeciwko gminie nie wchodziła już w grę możliwość wy\xaddania decyzji z art. 8 ust. 1 w/w ustawy w odniesieniu do wspomnianych praktyk monopolistycznych.\nRozpoznając w pozostałym zakresie odwołania wniesione przez najemców Sąd Antymonopolowy również nie znalazł podstaw do ich uwzględnienia. Nietrafne są w szczególności zarzuty odwołań, iż w zaskarżonej decyzji nie dostrzeżono praktyki monopolistycznej w prowadzeniu przez gminę przetargów w od\xadniesieniu do lokali, w stosunku do których najemcom przysłu\xadgują prawa wynikające z umowy najmu. Przeciwdziała tej prak\xadtyce pkt I zaskarżonej decyzji, gdzie nakazano gminie zanie\xadchanie odmowy zawierania umów najmu w formie pisemnej w od-r niesieniu do lokali udostępnianych najemcom na zasadzie bezu\xadmownego korzystania. Rozstrzygnięcie Urzędu Antymonopolowego zgodne jest tutaj z zasadą wyrażoną wart. 674 k.c.\nWprawdzie nie można podzielić poglądu zajętego w zaskar\xadżonej decyzji jakoby podejmowane przez gminę, w wykonaniu przepisów gminnych, działania zmierzające do kierowania loka\xadli użytkowych do przetargu nie podlegały ocenie ustawy anty\xadmonopolowej. Tym niemniej, zakładając uporządkowanie gospoda\xadrki lokalami użytkowymi w następstwie wykonania pktu I decy\xadzji, trudno dopatrzyć się w instytucji przetargu naruszenia przepisów ustawy antymonopolowej. Jeżeli do przetargu mogą na równych prawach przystąpić wszystkie zainteresowane podmioty gospodarcze, przetarg nie narusza celów ustawy antymonopolo\xadwej a w szczególności ochrony wolnej konkurencji.\nSąd Antymonopolowy nie podzielił stanowiska zajętego w odwołaniuA. M., w świetle którego różnicowanie przez gminę warunków umów najmu lokali użytkowych zlokalizo\xadwanych w centrum handlowym miasta w porównaniu do pozostałych lokali, stanowi podział rynku według kryterium terytorialnego\n\n  - 12\n\nlub asortymentowego w rozumieniu art. 5 pkt 2 ustawy antymo\xadnopolowej. Rynek w ujęciu przedmiotowym odnosi się do wszyst\xadkich towarów jednego rodzaju, które poprzez szczególne właś\xadciwości, w szczególności zaś przeznaczenie, użytkowość i cenę odróżniają się od innych towarów w taki sposób, że nie ist\xadnieje możliwość dowolnej ich zamiany. Rynek pod względem przedmiotowym (asortymentowym) tworzą więc towary, które z punktu widzenia nabywcy charakteryzują się substytutywnością. Pomimo więc podobnego pod względem przedmiotowym asortymentu, lokali użytkowych zlokalizowanych w centrum handlowym miasta w porównaniu do innych lokali użytkowych w tym mieście nie można, z punktu widzenia ogółu nabywców (najemców), uznać za substytutywne. Możliwości prowadzenia działalności gospodar\xadczej w lokalach znajdujących się w centrum handlowym miasta sa zdecydowanie korzystniejsze. Istotne jest także i to, że w warunkach gospodarki rynkowej obiektywną prawidłowością są znacznie wyższe czynsze najmu lokali użytkowych zlokalizowa\xadnych w centrach handlowych miast w porównaniu do pozostałych lokali. Uzasadnia to traktowanie takich lokali użytkowych ja\xadko odrębnego rynku. Z tego też względu stosowanie przez gminę innej polityki najmu w odniesieniu do tej kategorii lokali nic może być uznane jako podział rynku, lecz jako działania na dwóch odrębnych rynkach.\nBrak jest także podstaw do uwzględnienia zarzutu odwoła\xadnia upatrującego w odmowie sprzedaży lokali użytkowych ogra\xadniczanie przez gminę wolnej konkurencji na rynku w świetle art. 5 pkt 1 ustawy antymonopolowej, skoro nie stanowiło to przedmiotu rozstrzygnięcia zaskarżonej decyzji, a nawet przedmiotu żądania zgłoszonego we wniosku o wszczęcie postę\xadpowania. Formalnie rzecz biorąc nie ma przeszkód, aby wspom-\xad\n\n  - 13\n\nniany zarzut stanowił przedmiot odrębnego postępowania administracyjnego.\nNie znajdują także uzasadnienia, podnoszone w szczególności w postępowaniu administracyjnym, żądania skierowane pod adresem Urzędu Antymonopolowego, aby ten zobowiązał gminę do renegocjowania treści umów najmu z dotychczasowymi najemcami. W świetle art. 8 ust. 1 ustawy antymonopolowej takie oczeki\xadwania pod adresem organów działających w trybie wspomnianej ustawy, są za daleko idące. W postępowaniu administracyjnym toczącym się w trybie wspomnianej ustawy podmiotowi stosują\xadcemu praktyki monopolistyczne można jedynie nakazać ich za\xadniechanie. Nie można natomiast, w sposób pozytywny, nakazać podmiotowi spełnienie określonego zachowania mieszczącego się w granicach prawa. Reasumując zatem w trybie ustawy antymono\xadpolowej można wskazać podmiotowi gospodarczemu czego ma za\xadniechać. Natomiast nie wolno mu w prawnoprocesowych formach działania wskazać jak konkretnie ma się zachować w określonej sytuacji.\nW tym stanie sprawy Sąd Antymonopolowy odwołania oddalił (art. 4793 1§ 1 k.p.c.).\nO kosztach postępowania orzeczono na zasadzieart. 98 i 99 k.p.c.',
 'vol_number': 14.0,
 'vol_type': '15/450500/0005127/AmA',
 'court_name': 'Sąd Okręgowy w Warszawie',
 'department_name': 'XVII Wydział Sąd Ochrony Konkurencji i Konsumenta',
 'text_legal_bases': [{'address': 'Dz. U. z 1964 r. Nr 43, poz. 296',
   'art': 'art. 98;art. 99',
   'isap_id': 'WDU19640430296',
   'text': 'art. 98 i 99 k.p.c.',
   'title': 'Ustawa z dnia 17 listopada 1964 r. - Kodeks postępowania cywilnego'},
  {'address': 'Dz. U. z 1990 r. Nr 14, poz. 88',
   'art': 'art. 4;art. 4 ust. 1;art. 4 ust. 1 pkt. 1;art. 5',
   'isap_id': 'WDU19900140088',
   'text': 'art. 4 ust. 1 pkt 1 i art. 5 ustawy z dnia 24 lutego 1990 r. o przeciwdziałaniu praktykom monopolistycznym',
   'title': 'Ustawa z dnia 24 lutego 1990 r. o przeciwdziałaniu praktykom monopolistycznym'},
  {'address': 'Dz. U. z 1960 r. Nr 30, poz. 168',
   'art': 'art. 10/5;art. 10/5 § 1',
   'isap_id': 'WDU19600300168',
   'text': 'art. 10.5 §1 k.p.a.',
   'title': 'Ustawa z dnia 14 czerwca 1960 r. - Kodeks postępowania administracyjnego'},
  {'address': 'Dz. U. z 1964 r. Nr 16, poz. 93',
   'art': 'art. 674',
   'isap_id': 'WDU19640160093',
   'text': 'art. 674 k.c.',
   'title': 'Ustawa z dnia 23 kwietnia 1964 r. - Kodeks cywilny'}],
 'thesis': None}
```

</details>

### Data Fields

| Feature name     | Feature description                                                                              | Type                                             |
|------------------|--------------------------------------------------------------------------------------------------|--------------------------------------------------|
| _id              | unique identifier of the judgement                                                               | `string`  or  `None`                             |
| signature        | signature of judgement (unique within court)                                                     | `string` or  `None`                              |
| date             | date of judgement                                                                                | `datetime` (ISO format with timezone) or  `None` |
| publicationDate  | date of judgement publication                                                                    | `datetime` (ISO format with timezone) or  `None` |
| lastUpdate       | date of last update of judgement                                                                 | `datetime` (ISO format with timezone) or  `None` |
| courtId          | system unique identifier of the court                                                            | `int` or  `None`                                 |
| departmentId     | system unique identifier of the court's department                                               | `int` or  `None`                                 |
| type             | type of the judgement (one of )                                                                  | `string` or  `None`                              |
| excerpt          | First 500 characters of the judgement                                                            | `string` or  `None`                              |
| content          | Full content of judgement in XML format                                                          | `string` or `None`                               |
| chairman         | chairman judge name                                                                              | `string` or  `None`                              |
| decision         | decision                                                                                         | `string` or `None`                               |
| judges           | list of judge names participating in the judgement                                               | `list[string]` or  `None`                        |
| legalBases       | legal acts which are bases for the judgement                                                     | `list[dict[str, str]]` or  `None`                |
| publisher        | name of the person publishing the judgement                                                      | `string` or `None`                               |
| recorder         | name of the person recording the judgement                                                       | `string` or `None`                               |
| reviser          | name of the person revising the judgement                                                        | `string` or `None`                               |
| themePhrases     | list of phrases representing the themes/topics of the judgement                                  | `list[string]` or  `None`                        |
| num_pages        | number of pages in the judgement                                                                 | `float` or  `None`                               |
| text             | full text of the judgement                                                                       | `string` or `None`                               |
| vol_number       | volume number                                                                                    | `float` or  `None`                               |
| vol_type         | type of volume                                                                                   | `string` or `None`                               |
| court_name       | name of the court where the judgement was made                                                   | `string` or  `None`                              |
| department_name  | name of the department within the court where the judgement was made                             | `string` or  `None`                              |
| text_legal_bases | textual representation of the legal bases for the judgement (with references to online repository | `list[dict[str, str]]` or  `None`                |
| thesis           | thesis of the judgement                                                                          | `string` or `None`                               |


### Data Splits

This dataset is not split into subsets. The dataset has only `train` split.

## Dataset Creation

For details on the dataset creation, see the paper [TBA]() and the code repository [here](https://github.com/pwr-ai/JuDDGES).

### Curation Rationale

Created to enable cross-jurisdictional legal analytics.

### Source Data

#### Initial Data Collection and Normalization

1. Download judgements metadata.
1. Download judgements text (XML content of judgements).
1. Download additional details available for each judgement.
1. Map id of courts and departments to court name.
1. Extract raw text from XML content and details of judgments not available through API.
1. For further processing prepare local dataset dump in parquet file, version it with dvc and push to remote storage.

#### Who are the source language producers?

Produced by human legal professionals (judges, court clerks). Demographics was not analysed. Sourced from public court databases.

### Annotations

#### Annotation process

No annotation was performed by us. All features were provided via API (anonymization and publication of the data were performed by court employees).

#### Who are the annotators?

As above.

### Personal and Sensitive Information

Pseudoanonymized to comply with GDPR (art. 4 sec. 5 GDPR).

## Considerations for Using the Data

### Social Impact of Dataset

[More Information Needed]

### Discussion of Biases

[More Information Needed]

### Other Known Limitations

[More Information Needed]

## Additional Information

### Dataset Curators

[More Information Needed]

### Licensing Information

We license the actual packaging of these data under Attribution 4.0 International (CC BY 4.0) https://creativecommons.org/licenses/by/4.0/

### Citation Information

TBA

# Statistics

### Missing values
We identified 5,725 judgments (approximately 1%) with a missing content field. The root cause of these missing values is unknown and assumed to be due to random errors, as only error codes (e.g. 404) were observed when accessing the API during dataset curation. Given that this represents a very small fraction of the dataset, these missing values are not expected to impact the overall data quality, so we removed these judgments. Additionally, the table below displays the number and proportion of missing values across other fields in the dataset (after removing those with missing `content`).

In [ ]:
# | eval: false
null_count = (
    raw_ds.null_count().collect().to_pandas().T.rename(columns={0: "Null count"})
)
null_count.index.name = "Field name"
null_count["Null fraction"] = (
    null_count["Null count"] / raw_ds.select(pl.len()).collect().item()
).round(2)
# print(null_count.to_markdown())

| Field name       |   Null count |   Null fraction |
|:-----------------|-------------:|----------------:|
| _id              |            0 |            0    |
| signature        |            0 |            0    |
| date             |            0 |            0    |
| publicationDate  |            0 |            0    |
| lastUpdate       |            0 |            0    |
| courtId          |            0 |            0    |
| departmentId     |            0 |            0    |
| type             |            0 |            0    |
| excerpt          |            0 |            0    |
| content          |            0 |            0    |
| chairman         |        47283 |            0.12 |
| decision         |       408423 |            1    |
| judges           |        39772 |            0.1  |
| legalBases       |       113534 |            0.28 |
| publisher        |          609 |            0    |
| recorder         |       103675 |            0.25 |
| references       |        40737 |            0.1  |
| reviser          |          171 |            0    |
| themePhrases     |       117074 |            0.29 |
| num_pages        |            0 |            0    |
| text             |            0 |            0    |
| vol_number       |            0 |            0    |
| vol_type         |            0 |            0    |
| court_name       |          605 |            0    |
| department_name  |          605 |            0    |
| text_legal_bases |            0 |            0    |
| thesis           |       369092 |            0.9  |

### Analysis of selected fields

In [ ]:
# | eval: false
court_distribution = (
    raw_ds.drop_nulls(subset="court_name")
    .select("court_name")
    .group_by("court_name")
    .len()
    .sort("len", descending=True)
    .collect()
    .to_pandas()
)
ax = sns.histplot(data=court_distribution, x="len", log_scale=True, kde=True)
ax.set(
    title="Distribution of judgments per court",
    xlabel="#Judgements in single court",
    ylabel="Count",
)
plt.show()

In [ ]:
# | eval: false
judgements_per_year = (
    raw_ds.select("date")
    .collect()["date"]
    .str.split(" ")
    .list.get(0)
    .str.to_date()
    .dt.year()
    .value_counts()
    .sort("date")
    .to_pandas()
)
judgements_per_year = judgements_per_year[judgements_per_year["date"] < 2024]

_, ax = plt.subplots(1, 1, figsize=(10, 5))
ax = sns.pointplot(
    data=judgements_per_year, x="date", y="count", linestyles="--", ax=ax
)
ax.set(
    xlabel="Year",
    ylabel="Number of Judgements",
    title="Yearly Number of Judgements",
    yscale="log",
)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# | eval: false
types = (
    raw_ds.fill_null(value="<null>")
    .select("type")
    .group_by("type")
    .len()
    .sort("len", descending=True)
    .collect()
    .to_pandas()
)

_, ax = plt.subplots(1, 1, figsize=(8, 8))
ax = sns.barplot(data=types, x="len", y="type", errorbar=None, ax=ax)
ax.set(xlabel="Count", ylabel="Type", title="Judgement types cardinality", xscale="log")
plt.show()

In [ ]:
# | eval: false
num_judges = (
    raw_ds.with_columns([pl.col("judges").list.len().alias("num_judges")])
    .select("num_judges")
    .sort("num_judges")
    .collect()
    .to_pandas()
)
ax = sns.histplot(
    data=num_judges, x="num_judges", bins=num_judges["num_judges"].nunique()
)
ax.set(
    xlabel="#Judges per judgement",
    ylabel="Count",
    yscale="log",
    title="#Judges per single judgement",
)
plt.show()

In [ ]:
# | eval: false
num_lb = (
    raw_ds.with_columns([pl.col("legalBases").list.len().alias("num_lb")])
    .select("num_lb")
    .sort("num_lb")
    .collect()
    .to_pandas()
)
ax = sns.histplot(data=num_lb, x="num_lb", bins=num_lb["num_lb"].nunique())
ax.set(
    xlabel="#Legal bases",
    ylabel="Count",
    yscale="log",
    title="#Legal bases per judgement",
)
plt.show()

In [ ]:
# | eval: false
raw_text_ds = load_dataset(
    "parquet", data_dir="../../data/datasets/pl/raw/", columns=["_id", "text"]
)
raw_text_ds = raw_text_ds.filter(lambda x: x["text"] is not None)

In [ ]:
# | eval: false
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")


def tokenize(batch: dict[str, list]) -> list[int]:
    tokenized = tokenizer(
        batch["text"],
        add_special_tokens=False,
        return_attention_mask=False,
        return_token_type_ids=False,
        return_length=True,
    )
    return {"length": tokenized["length"]}


raw_text_ds = raw_text_ds.map(
    tokenize, batched=True, batch_size=16, remove_columns=["text"], num_proc=20
)

In [ ]:
# | eval: false
judgement_len = raw_text_ds["train"].to_pandas()

ax = sns.histplot(data=judgement_len, x="length", bins=50)
ax.set(
    xlabel="#Tokens",
    ylabel="Count",
    title="#Tokens distribution in judgements (llama-3 tokenizer)",
    yscale="log",
)
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f"{int(x/1_000)}k"))
plt.show()

In [ ]:
# | eval: false
per_type_tokens = (
    raw_ds.fill_null(value="<null>")
    .select(["_id", "type"])
    .collect()
    .to_pandas()
    .set_index("_id")
    .join(judgement_len.set_index("_id"))
)

_, ax = plt.subplots(1, 1, figsize=(10, 10))
ax = sns.boxenplot(data=per_type_tokens, y="type", x="length")
ax.set(
    xscale="log",
    title="Judgement token count per type",
    xlabel="#Tokens",
    ylabel="Type",
)
plt.show()